In [27]:
!pip install langchain-groq # install the langchain_groq module
from langchain_groq import ChatGroq # import the ChatGroq class from the langchain_groq module

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.1 MB/s eta 0:00:00


In [28]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

In [29]:
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [37]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38322")
page_data = loader.load().pop().page_content
print(page_data)

Apply for NIKE, Inc. Product Management Graduate Internship

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE, Inc. Product Management Graduate InternshipBeaverton, Oregon

In [50]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

{
  "role": "NIKE, Inc. Product Management Graduate Internship",
  "experience": "3+ years",
  "skills": [
    "Product management",
    "Strategy",
    "Engineering",
    "Web and Mobile software development",
    "Data analysis",
    "Product strategy",
    "Communication",
    "Collaboration",
    "Problem-solving",
    "Leadership"
  ],
  "description": "As the Product Management Intern, you will work with Product Management teams across Nike, in one of the following teams: Digital, Marketplace Partners, Supply Chain, Planning or Enterprise Architecture and Developer Platforms (EADP). You will orchestrate an exciting internship project, giving you the experience and ability to be a transformative leader, problem solver, collaborator, and teammate with Nike leaders to setup Nike to deliver business and/or consumer facing products that are viable and feasible to our Business and valuable and loved by our Consumers."
}


In [51]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'role': 'NIKE, Inc. Product Management Graduate Internship', 'experience': '3+ years', 'skills': ['Product management', 'Strategy', 'Engineering', 'Web and Mobile software development', 'Data analysis', 'Product strategy', 'Communication', 'Collaboration', 'Problem-solving', 'Leadership'], 'description': 'As the Product Management Intern, you will work with Product Management teams across Nike, in one of the following teams: Digital, Marketplace Partners, Supply Chain, Planning or Enterprise Architecture and Developer Platforms (EADP). You will orchestrate an exciting internship project, giving you the experience and ability to be a transformative leader, problem solver, collaborator, and teammate with Nike leaders to setup Nike to deliver business and/or consumer facing products that are viable and feasible to our Business and valuable and loved by our Consumers.'}


In [52]:
type(json_res)

dict

In [55]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [57]:
# !pip install chromadb
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
        metadatas={"links":row["Links"]},
        ids=[str(uuid.uuid4())])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.9 MB/s eta 0:00:0

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 40.1MiB/s]


In [69]:
links = collection.query(query_texts=job['skills'], n_results=2).get("metadatas",[])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https

In [70]:
job = json_res
job['skills']

['Product management',
 'Strategy',
 'Engineering',
 'Web and Mobile software development',
 'Data analysis',
 'Product strategy',
 'Communication',
 'Collaboration',
 'Problem-solving',
 'Leadership']

In [72]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Salman, a business development executive at Axle. Axle is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Axle
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Axle's portfolio: {link_list}
        Remember you are Salman, BDE at Axle.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Product Management Solutions for Nike, Inc.

Dear Hiring Manager,

I came across the Product Management Graduate Internship role at Nike, Inc. and was impressed by the company's commitment to innovation and excellence. As a Business Development Executive at Axle, I'd like to introduce our company as a potential partner in helping Nike achieve its product management goals.

At Axle, we specialize in providing AI and software consulting services that facilitate seamless integration of business processes through automated tools. Our expertise in product management, strategy, and software development aligns perfectly with the requirements of the internship project.

Our team has a proven track record of delivering scalable, efficient, and cost-effective solutions to numerous enterprises. We'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* Magento-based e-commerce solutions: https://example.com/magento-portfolio
* Machine L